In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
#!pip install pandas --upgrade
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
df = pd.read_csv('crimes.csv')
print("finished loading")

In [ ]:
import datetime
import locale

store = pd.HDFStore('store.h5')


data_subset = pd.DataFrame(df.subset(20000).Date)

# add columns
data_subset.insert(0, 'Hour', 0)
data_subset.insert(0, 'Day', 0)
data_subset.insert(0, 'Month', 0)
data_subset.insert(0, 'Year', 0)

# FORMAT : 06/25/2012 03:00:00 PM

# convert string date time to 24 hour (just hour)
# output as string
def to24Hour(x):
    hour = x[11:13]
    am = x[-2:] == 'AM'
    if(am):
        if(hour == '12'):
            hour = '00'
        else:
            return hour
    else:
        if(hour != '12'):
            hour = str(int(hour) + 12)

    return hour

# convert string to datetime object with only date
def toDate(x):
    month = x[0:2]
    date = x[3:5]
    year = x[6:10]
    return datetime.datetime(int(year), int(month), int(date))

def toMonth(x):
    return x[0:2]

def toYear(x):
    return x[6:10]
    
    
    


# convert date
locale.setlocale(locale.LC_ALL, '')
#datetime.datetime.strptime('06/25/2012 03:00:00 PM', '%x %X %p').strftime('%A')
#https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
data_subset['Hour'] = data_subset['Date'].apply(to24Hour)
data_subset['Day'] = data_subset['Date'].apply(lambda x: toDate(x).strftime('%w'))
data_subset['Month'] = data_subset['Date'].apply(toMonth)
data_subset['Year'] = data_subset['Date'].apply(toYear)

# save to HDF5
store['data_subset'] = data_subset
#retrieve with varName = store['data_subset']

store.close()

data_subset



In [11]:
import matplotlib.pyplot as plt

In [12]:
hourCounts = data_subset.Hour.value_counts().sort_index()

X = np.arange(len(hourCounts))

plt.bar(X, hourCounts.values)

plt.xticks(X, hourCounts.axes[0])

plt.show()

In [171]:
dayCounts = data_subset.Day.value_counts().sort_index()

X = np.arange(len(dayCounts))

plt.bar(X, dayCounts.values, align='center')

plt.xticks(X, dayCounts.axes[0]) # ['S', 'M', 'T', 'W', 'Th', 'F', 'Sa'])#

plt.show()

In [173]:
monthCounts = data_subset.Month.value_counts().sort_index()

X = np.arange(len(monthCounts))

plt.bar(X, monthCounts.values, align='center')

plt.xticks(X, monthCounts.axes[0]) # ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.show()

In [175]:
yearCounts = data_subset.Year.value_counts().sort_index()

X = np.arange(len(yearCounts))

plt.bar(X, yearCounts.values, align='center')

plt.xticks(X, yearCounts.axes[0])

plt.show()